<a href="https://colab.research.google.com/github/Jarvis-BITS/MIT-midodrine/blob/main/Midodrine_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
print("Tensorflow version " + tf.__version__)


Tensorflow version 2.8.0


run if TPU's are *connected*

In [ ]:
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

run if GPU's connected

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

# Demographic analysis


---


**List of variables/column names we should pull**: 

1.   from core: subject_id, gender, admittime,	dischtime, deathtime, ethnicity
2.   from derived: weight/height, age, some indicators related to midodrine injection (still in search)


